In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import wordcloud

In [ ]:
data = pd.read_csv("../input/indian-food-101/indian_food.csv")

In [ ]:
print(data.info())

In [ ]:
data.head(5)

In [ ]:
print("Total different dishes: {}".format(data.name.nunique()))

In [ ]:
data['state'] = data['state'].apply(lambda x: x if x!="-1" else "Unknown State")
data['flavor_profile'] = data['flavor_profile'].apply(lambda x: x if x!="-1" else "Unknown Flavor")

In [ ]:
state_cnt = data['name'].groupby(data['state']).aggregate('count').sort_values()
flv_cnt = data['name'].groupby(data['flavor_profile']).aggregate('count').sort_values()
course_cnt = data['name'].groupby(data['course']).aggregate('count').sort_values()
diet_cnt = data['name'].groupby(data['diet']).aggregate('count').sort_values()

# Distributions for Dishes

In [ ]:
# Distribution of dishes
clr = '#0c0052'
fig = plt.figure(figsize=(15, 18))
fig.add_subplot(3,2,1)
sns.barplot(y=state_cnt.index[::-1], x=state_cnt[::-1], color=clr)
sns.despine(right=True, top=True)
plt.xlabel('Number of Dishes')
plt.ylabel('State')
plt.title("Statewise distribution for dishes")

fig.add_subplot(3,2,2)
sns.barplot(y=flv_cnt.index[::-1], x=flv_cnt[::-1], color=clr)
sns.despine(right=True, top=True)
plt.xlabel('Number of Dishes')
plt.ylabel('Flavor')
plt.title("Flavor distribution for dishes")

fig.add_subplot(3,2,3)
sns.barplot(y=flv_cnt.index[::-1], x=flv_cnt[::-1], color=clr)
sns.despine(right=True, top=True)
plt.xlabel('Number of Dishes')
plt.ylabel('Flavor')
plt.title("Flavor distribution for dishes")

fig.add_subplot(3,2,3)
sns.barplot(y=course_cnt.index[::-1], x=course_cnt[::-1], color=clr)
sns.despine(right=True, top=True)
plt.xlabel('Number of Dishes')
plt.ylabel('Course')
plt.title("Course type distribution for dishes")

fig.add_subplot(3,2,4)
sns.barplot(x=diet_cnt.index[::-1], y=diet_cnt[::-1], color=clr)
sns.despine(right=True, top=True)
plt.xlabel('Number of Dishes')
plt.ylabel('Diet')
plt.title("Diet type distribution for dishes")

fig.add_subplot(3,2,5)
sns.barplot(y='name', x='prep_time', data=data[['name', 'prep_time']].sort_values(by='prep_time')[::-1][:15], color=clr)
sns.despine(right=True, top=True)
plt.ylabel('Dish')
plt.xlabel('Preparation Time')
plt.title("Top 15 dishes with most preparation time")


fig.add_subplot(3,2,6)
sns.barplot(y='name', x='cook_time', data=data[['name', 'cook_time']].sort_values(by='cook_time')[::-1][:15], color=clr)
sns.despine(right=True, top=True)
plt.ylabel('Dish')
plt.xlabel('Cooking Time')
plt.title("Top 15 dishes with most cooking time")

fig.tight_layout()
plt.show()


In [ ]:
all_ingredients = ",".join(data.ingredients.tolist()).split(',')
all_ingredients_s = pd.Series(all_ingredients, name='All Ingredients')
total_unique_ingredients = list(set(all_ingredients))

In [ ]:
print("Unique ingredients count : {}".format(len(total_unique_ingredients)))

In [ ]:
ing_c = all_ingredients_s.groupby(all_ingredients_s).aggregate('count').sort_values()
lbls = [i.strip() for i in ing_c.index]
fig = plt.figure(figsize=(5, 9))
sns.barplot(y=lbls[::-1][:30], x=ing_c.values[::-1][:30], color="black")
sns.despine(left=True, bottom=True)
plt.title("Top 30 most used ingredients")
plt.xlabel("Number of dishes")
plt.ylabel("Ingredient")
plt.plot()

# Distributions for Ingredients

In [ ]:
def create_ingredient_series(list_of_ingredients):
  all_ingredients = ",".join(list_of_ingredients).split(',')
  all_ingredients = [i.lower().strip() for i in all_ingredients]
  #print(all_ingredients)
  all_ingredients_s = pd.Series(all_ingredients)
  ing_c = all_ingredients_s.groupby(all_ingredients_s).aggregate('count').sort_values(ascending=False)
  return ing_c

# Diet wise distribution for Ingredients

In [ ]:
d_ing = []
for d in list(set(data['diet'])):
  diet_ing = create_ingredient_series(data[data['diet']==d]['ingredients'].tolist())
  d_ing.append(diet_ing)

fig = plt.figure(figsize=(12, 5))
fig.add_subplot(1, 2, 1)
sns.barplot(y=d_ing[1].index[:15], x=d_ing[1].values[:15], color="black")
sns.despine(left=True, bottom=True)
plt.title("Top 15 most used ingredients in Vegetarian dishes")
plt.xlabel("Number of dishes")
fig.add_subplot(1, 2, 2)
sns.barplot(y=d_ing[0].index[:15], x=d_ing[0].values[:15], color="black")
sns.despine(left=True, bottom=True)
plt.title("Top 15 most used ingredients in Non - Vegetarian dishes")
plt.xlabel("Number of dishes")
fig.tight_layout()
plt.show()

# State wise distribution for Ingredients

In [ ]:
st_ing = []
st_lst = data['name'].groupby(data['state']).aggregate('count').sort_values(ascending=False).index[:5]
for f in st_lst:
  stat_ing = create_ingredient_series(data[data['state']==f]['ingredients'].tolist())
  st_ing.append(stat_ing)

fig = plt.figure(figsize=(12, 15))
for i in range(len(st_ing)):
  fig.add_subplot(3, 2, i+1)
  sns.barplot(y=st_ing[i].index[:10], x=st_ing[i].values[:10], color="black")
  sns.despine(left=True, bottom=True)
  plt.title("Top 10 most used ingredients in {}".format(st_lst[i]))
  plt.xlabel("Number of dishes")
  plt.ylabel("Ingredient")
fig.tight_layout()
plt.show()

# Flavor wise distribution for Ingredients

In [ ]:
f_ing = []
flav_lst = list(set(data['flavor_profile']))
for f in flav_lst:
  flv_ing = create_ingredient_series(data[data['flavor_profile']==f]['ingredients'].tolist())
  f_ing.append(flv_ing)

fig = plt.figure(figsize=(16, 10))
for i in range(len(f_ing)):
  fig.add_subplot(2, 3, i+1)
  sns.barplot(y=f_ing[i].index[:15], x=f_ing[i].values[:15], color="black")
  sns.despine(left=True, bottom=True)
  plt.title("Top 15 most used ingredients in {} dishes".format(flav_lst[i]))
  plt.xlabel("Number of dishes")
fig.tight_layout()
plt.show()

Work in progress.